<a href="https://colab.research.google.com/github/yavuzuzun/projects/blob/main/kaggle_campaign_contribution_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this Kaggle challange, I predicted the winning probabilities of candidates in a former US election based on general statistics of the election on each region and the network activity between contributors and candidates.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Import traning set, test set, and network data.

In [ ]:
import pandas as pd
campaign = pd.read_csv('training_data.csv')
campaign_test = pd.read_csv('test_data.csv')
contributers_fed = pd.read_csv('federal_contributor_top100_contributors_network.csv')
contributers_sta = pd.read_csv('state_contributor_top100_contributors_network.csv')
candidates = pd.read_csv('all_candidates_state_bipartite_weighted_network.csv')
winners_sta = pd.read_csv('winning_candidates_state_bipartite_weighted_network.csv')

<ipython-input-18-5355d75e2244>:2: DtypeWarning: Columns (31,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  campaign = pd.read_csv('training_data.csv')


Clean data and prepare the traning set for model fitting

In [ ]:
X_train = campaign.drop(['general_sector','winner_ratio','city','zip_code','specific_sector',\
                                          'state','contributor_type'], axis=1)
y_train = campaign['winner_ratio']

X_train_encoded = X_train

# replace infinities with nan
X_train_encoded.replace([np.inf, -np.inf], np.nan, inplace=True)

for column in X_train_encoded.columns:
  # replace non-fitting values with the mode
  mode_value = X_train_encoded[column].mode().iloc[0]
  X_train_encoded[column].fillna(mode_value, inplace=True)
  X_train_encoded[column].replace(['#NAME?'], mode_value, inplace=True)

# convert non-float columns to float
non_float_columns = X_train.select_dtypes(exclude=['float']).columns.tolist()
for column in non_float_columns:
  if X_train[column].dtype == 'O':
    X_train_encoded[column] = X_train[column].values
    X_train_encoded[column] = X_train_encoded[column].astype(float)
    continue
  X_train_encoded[column] = X_train[column].astype(float)

X_train_encoded

,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,politician_democratic_count,politician_incumbency_count,...,governor_contribution_ratio,house_and_assembly_contribution_ratio,politician_challenger_ratio,politician_democratic_ratio,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio
0,4.0,4.0,0.0,6.0,6.0,0.0,0.0,4.0,4.0,2.0,...,0.50000,0.000000,0.0,1.0,0.5,0.5,0.0,0.000000,0.0,0.0
1,2.0,2.0,0.0,2.0,2.0,0.0,1.0,2.0,2.0,1.0,...,0.00000,0.216202,0.5,1.0,0.5,0.0,0.0,0.000000,0.0,0.0
2,4.0,4.0,0.0,4.0,4.0,0.0,0.0,4.0,4.0,4.0,...,0.00000,3.257082,0.0,1.0,1.0,0.0,0.0,-2.257082,0.0,0.0
3,4.0,4.0,0.0,4.0,4.0,0.0,0.0,4.0,4.0,2.0,...,0.60000,0.200000,0.0,1.0,0.5,0.5,0.0,0.200000,0.0,0.0
4,4.0,4.0,0.0,6.0,6.0,0.0,2.0,4.0,4.0,0.0,...,1.00000,0.000000,0.5,1.0,0.0,0.5,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172843,2.0,0.0,2.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,...,0.00000,0.000000,0.0,0.0,1.0,0.0,1.0,1.000000,0.0,0.0
172844,2.0,2.0,0.0,4.0,4.0,0.0,2.0,2.0,2.0,0.0,...,0.00000,1.000000,1.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.0
172845,2.0,0.0,2.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,...,0.00000,0.000000,0.0,0.0,1.0,0.0,1.0,1.000000,0.0,0.0
172846,2.0,0.0,2.0,2.0,0.0,2.0,0.0,2.0,0.0,0.0,...,1.00000,0.000000,0.0,0.0,0.0,1.0,1.0,0.000000,0.0,0.0


Use random forest regressor since the data is more likely to be categorical and RFR is more prone to overfitting.

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

# Train a random forest regressor on the training set
model = RandomForestRegressor(n_estimators=100, max_depth=50, random_state=333)
model.fit(X_train_encoded, y_train)

# Make predictions on the training set
y_pred = model.predict(X_train_encoded)

# Calculate the r-squared value
RMSE = mean_squared_error(y_train, y_pred)
print(f"R-squared: {RMSE:.3f}")

R-squared: 0.044


Clean and pre-process the test data

In [ ]:
X_test = pd.DataFrame()
for column in X_train_encoded.columns:
  if column in campaign_test.columns:
    X_test[column] = campaign_test[column]
  else:
    X_test[column] = np.zeros([X_test.shape[0],1])

X_test_encoded = X_test
X_test_encoded.replace([np.inf, -np.inf], np.nan, inplace=True)
for column in X_test_encoded.columns:
  # replace non-fitting values with the mode
  mode_value = X_test_encoded[column].mode().iloc[0]
  X_test_encoded[column].fillna(mode_value, inplace=True)
  X_test_encoded[column].replace(['#NAME?'], mode_value, inplace=True)

# convert non-float columns to float
non_float_columns = X_test.select_dtypes(exclude=['float']).columns.tolist()

for column in non_float_columns:
  if X_test_encoded[column].dtype == 'O':
    X_test_encoded[column] = X_test[column].values
    X_test_encoded[column] = X_test_encoded[column].astype(float)
    continue
  X_train_encoded[column] = X_train[column].astype(float)

In [ ]:
y_pred_test = model.predict(X_test_encoded)

Download the results

In [ ]:
from google.colab import files

submission_df = pd.DataFrame({'index': np.arange(len(y_pred_test)),'winner_ratio': y_pred_test}) 

submission_df.to_csv('y_pred_test_DT_35.csv', index=False)
files.download('y_pred_test_DT_35.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Repeat the same procedure for dwcision trees and compare the results. I expect a higher rmse since decision trees regressor is more likely to overfit the training set.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Train a random forest regressor on the training set
model = DecisionTreeRegressor(max_depth=35, random_state=333)
model.fit(X_train_encoded, y_train)

# Make predictions on the training set
y_pred = model.predict(X_train_encoded)

# Calculate the r-squared value
RMSE = mean_squared_error(y_train, y_pred)
print(f"R-squared: {RMSE:.3f}")

R-squared: 0.039


In [ ]:
y_pred_test = model.predict(X_test_encoded)

In [ ]:
from google.colab import files

submission_df = pd.DataFrame({'index': np.arange(len(y_pred_test)),'winner_ratio': y_pred_test}) 

submission_df.to_csv('y_pred_test_DT_35.csv', index=False)
files.download('y_pred_test_DT_35.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Resulting rmse was higher for decision trees which we expected.

# Improve the dataset by adding centrality values of the contributers for a given state 

In [ ]:
candidates_all = candidates.iloc[:,0].tolist()
winners = winners_sta.iloc[:, 0].tolist()
contributers_f = contributers_fed.iloc[:,0].tolist()
contributers_s = contributers_sta.iloc[:,0].tolist()

Process data for transfering to the networkx environment

In [ ]:
new_columns = [col.split()[-1] for col in contributers_fed.columns]
contributers_fed.columns = new_columns
contributers_fed['0'] = new_columns[1:]
contributers_fed

,0,NY,IA,NJ,IL,NY,FL,TX,DC,DC,...,TX,DC,DC,CO,DC,FL,NY,DC,TX,FL
0,NY,0.0,0.00,0.00,0.000000e+00,0.00,0.00,0.00,0.000000e+00,0.000000e+00,...,0.00,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.00,0.000000e+00,0.000000e+00,0.00,0.00
1,IA,0.0,0.00,0.00,1.552678e+05,0.00,0.00,0.00,0.000000e+00,6.387783e+05,...,0.00,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.00,0.000000e+00,7.107440e+03,0.00,0.00
2,NJ,0.0,0.00,0.00,8.813676e+05,0.00,0.00,0.00,0.000000e+00,0.000000e+00,...,0.00,8.813676e+05,0.000000e+00,0.00,8.813676e+05,0.00,0.000000e+00,0.000000e+00,0.00,0.00
3,IL,0.0,155267.77,881367.65,0.000000e+00,8189888.49,12744343.03,353824.67,7.778097e+07,5.607803e+07,...,4731502.26,1.023860e+08,1.102836e+08,2225867.27,1.332554e+08,17269919.51,2.315478e+07,9.943879e+07,10690278.85,1231252.97
4,NY,0.0,0.00,0.00,8.189888e+06,0.00,1681576.00,0.00,3.669357e+06,0.000000e+00,...,355389.42,1.241871e+07,3.032412e+06,0.00,1.487489e+07,347110.75,4.770090e+06,2.863865e+06,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,FL,0.0,0.00,0.00,1.726992e+07,347110.75,3656599.83,0.00,9.213115e+06,2.869731e+06,...,421440.48,1.003164e+07,1.565473e+07,0.00,1.243221e+07,0.00,3.463707e+06,7.933603e+06,725780.26,490253.58
196,NY,0.0,0.00,0.00,2.315478e+07,4770089.67,9952311.55,0.00,9.130144e+06,0.000000e+00,...,97102.64,2.499765e+07,1.325871e+07,0.00,2.614165e+07,3463707.36,0.000000e+00,1.125702e+07,0.00,648483.08
197,DC,0.0,7107.44,0.00,9.943879e+07,2863865.04,4031875.49,0.00,5.682686e+07,3.788182e+07,...,4637205.39,3.826197e+07,6.948174e+07,2619885.75,5.097495e+07,7933602.63,1.125702e+07,0.000000e+00,6076750.61,740999.39
198,TX,0.0,0.00,0.00,1.069028e+07,0.00,0.00,0.00,4.363170e+07,6.292511e+06,...,842661.19,3.544950e+05,8.837380e+06,721227.98,3.870490e+05,725780.26,0.000000e+00,6.076751e+06,0.00,0.00


Federal contributers according to states

In [ ]:
import networkx as nx
array = contributers_fed.drop('0',axis=1)
array_fed = array.to_numpy()
G_fed = nx.from_numpy_array(array_fed)
EC_fed = nx.eigenvector_centrality(G_fed)
LC_fed = nx.laplacian_centrality(G_fed)

In [ ]:
EC_f = {}
index = 1
for key, value in EC_fed.items():
    EC_f[new_columns[index]] = value
    index = index + 1

LC_f = {}
index = 1
for key, value in LC_fed.items():
    LC_f[new_columns[index]] = value
    index = index + 1

State contributers according to states

In [ ]:
new_columns_s = [col.split()[-1] for col in contributers_fed.columns]
contributers_sta.columns = new_columns_s
contributers_sta['0'] = new_columns_s[1:]
contributers_sta

,0,NY,IA,NJ,IL,NY,FL,TX,DC,DC,...,TX,DC,DC,CO,DC,FL,NY,DC,TX,FL
0,NY,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
1,IA,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,...,0.0,420271.68,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
2,NJ,0.0,0.0,0.0,0.0,0.0,0.00,0.0,154802.86,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
3,IL,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
4,NY,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,FL,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
196,NY,0.0,0.0,0.0,0.0,0.0,0.00,0.0,937522.70,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,1121589.81
197,DC,0.0,0.0,0.0,0.0,0.0,120053.19,0.0,0.00,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
198,TX,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00


In [ ]:
array = contributers_sta.drop('0',axis=1)
array_sta = array.to_numpy()
G_sta = nx.from_numpy_array(array_sta)
EC_sta = nx.eigenvector_centrality(G_sta)
LC_sta = nx.laplacian_centrality(G_sta)

In [ ]:
EC_s = {}
index = 1
for key, value in EC_sta.items():
    EC_s[new_columns_s[index]] = value
    index = index + 1

LC_s = {}
index = 1
for key, value in LC_sta.items():
    LC_s[new_columns_s[index]] = value
    index = index + 1

In [ ]:
# Create a new column using the dictionary
X_train_encoded['eig_fed'] = campaign['state'].map(EC_f)
X_train_encoded['lap_fed'] = campaign['state'].map(LC_f)
X_train_encoded['eig_sta'] = campaign['state'].map(EC_s)
X_train_encoded['lap_sta'] = campaign['state'].map(LC_s)
X_test_encoded['eig_fed'] = campaign_test['state'].map(EC_f)
X_test_encoded['lap_fed'] = campaign_test['state'].map(LC_f)
X_test_encoded['eig_sta'] = campaign_test['state'].map(EC_s)
X_test_encoded['lap_sta'] = campaign_test['state'].map(LC_s)

In [ ]:
# retrain the model with updated columns

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

# Train a random forest regressor on the training set
model = RandomForestRegressor(n_estimators=100, max_depth=25, random_state=333)

# replace infinities with nan
X_train_encoded.replace([np.inf, -np.inf], np.nan, inplace=True)

for column in X_train_encoded.columns:
  # replace non-fitting values with the mode
  mode_value = X_train_encoded[column].mode().iloc[0]
  X_train_encoded[column].fillna(mode_value, inplace=True)
  X_train_encoded[column].replace(['#NAME?'], mode_value, inplace=True)

model.fit(X_train_encoded, y_train)

# Make predictions on the training set
y_pred = model.predict(X_train_encoded)

# Calculate the r-squared value
RMSE = mean_squared_error(y_train, y_pred)
print(f"R-squared: {RMSE:.3f}")

R-squared: 0.031


In [ ]:
# clean test data columns

# replace infinities with nan
X_test_encoded.replace([np.inf, -np.inf], np.nan, inplace=True)

for column in X_test_encoded.columns:
  # replace non-fitting values with the mode
  mode_value = X_test_encoded[column].mode().iloc[0]
  X_test_encoded[column].fillna(mode_value, inplace=True)
  X_test_encoded[column].replace(['#NAME?'], mode_value, inplace=True)

In [ ]:
# test the data with the model trained with the new centrality columns
y_pred_test = model.predict(X_test_encoded)

In [ ]:
from google.colab import files

submission_df = pd.DataFrame({'index': np.arange(len(y_pred_test)),'winner_ratio': y_pred_test}) 

submission_df.to_csv('y_pred_test_DT_25.csv', index=False)
files.download('y_pred_test_DT_25.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>